In [1]:
import pandas as pd
import numpy as np

In [2]:
# ---------- Step 1: Load and clean master rainfall dataset ----------
rain = pd.read_csv(r"/content/rain.csv")
rain['date'] = pd.to_datetime(rain['date'], errors='coerce')
rain_cols = ["rfh", "rfh_avg", "r1h", "r1h_avg", "r3h", "r3h_avg", "rfq", "r1q", "r3q"]
rain[rain_cols] = rain[rain_cols].apply(pd.to_numeric, errors='coerce')

# Keep only relevant columns to save memory
rain_small = rain[['date'] + rain_cols]
rain_small.to_csv("rain_intermediate.csv", index=False)


/tmp/ipython-input-2112616999.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rain['date'] = pd.to_datetime(rain['date'], errors='coerce')


In [5]:
# Read rainfall intermediate and ensure date is datetime
df = pd.read_csv("rain_intermediate.csv")
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # convert to datetime

# Read temperature
temp_pk = pd.read_csv(r"/content/temp_pk.csv")
temp_pk.columns = temp_pk.columns.str.strip()  # remove spaces
temp_pk['date'] = pd.to_datetime(temp_pk['Year'].astype(str) + "-" + temp_pk['Month'] + "-01",
                                 errors='coerce')

temp_pk_small = temp_pk[['date', 'Temperature - (Celsius)']]

# Merge safely
df = df.merge(temp_pk_small, on='date', how='left')
df.to_csv("merged_step1.csv", index=False)

print("Merged Step 1 complete")


Merged Step 1 complete


Loading all datasets

In [7]:
# ---------- Step 3: Merge Area Sown ----------
area_sown = pd.read_csv(r"/content/area_sown.csv")
area_sown.rename(columns={"Province/division\n/district": "Province"}, inplace=True)
area_sown_long = area_sown.melt(id_vars=["Province"], var_name="Year", value_name="Area_Sown")
area_sown_long["Year"] = area_sown_long["Year"].str.split("-").str[0].astype(int)
area_sown_long["date"] = pd.to_datetime(area_sown_long["Year"].astype(str) + "-01-01")
area_sown_long_small = area_sown_long[['date', 'Area_Sown']]

# Read the intermediate merged dataset
df = pd.read_csv("merged_step1.csv")

# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Ensure area_sown date column is also datetime
area_sown_long_small['date'] = pd.to_datetime(area_sown_long_small['date'], errors='coerce')

# Merge safely
df = df.merge(area_sown_long_small, on='date', how='left')
df.to_csv("merged_step2.csv", index=False)

print("Merged Step 2 complete")


/tmp/ipython-input-4015671740.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  area_sown_long_small['date'] = pd.to_datetime(area_sown_long_small['date'], errors='coerce')


Merged Step 2 complete


In [9]:
# Read intermediate merged dataset
df = pd.read_csv("merged_step2.csv")

# Convert 'date' column to datetime (fix object vs datetime mismatch)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Read wheat percentage dataset
wheat_pct = pd.read_csv(r"/content/wheat_pct.csv")

# Fix column names
wheat_pct.rename(columns={"Province/division\n/district": "Province"}, inplace=True)

# Melt to long format
wheat_pct_long = wheat_pct.melt(
    id_vars=["Province"],
    var_name="Year",
    value_name="Wheat_Percentage"
)

# Fix year format (take first year of range)
wheat_pct_long["Year"] = wheat_pct_long["Year"].str.split("-").str[0].astype(int)

# Generate datetime column for merge
wheat_pct_long["date"] = pd.to_datetime(wheat_pct_long["Year"].astype(str) + "-01-01", errors='coerce')

# Keep only necessary columns to save memory
wheat_pct_long_small = wheat_pct_long[['date', 'Wheat_Percentage']]

# Convert wheat_pct date column to datetime (just to be safe)
wheat_pct_long_small['date'] = pd.to_datetime(wheat_pct_long_small['date'], errors='coerce')

# Merge with existing dataset
df = df.merge(wheat_pct_long_small, on='date', how='left')

# Save
df.to_csv("merged_step3.csv", index=False)

print("Merged Step 3 (Wheat Percentage) complete")

/tmp/ipython-input-776303070.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wheat_pct_long_small['date'] = pd.to_datetime(wheat_pct_long_small['date'], errors='coerce')


Merged Step 3 (Wheat Percentage) complete


In [ ]:
# ---------- Step 5: Merge Crop Yield ----------
# Read intermediate merged dataset
df = pd.read_csv("merged_step3.csv")

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Read crop yield dataset
crop_yield = pd.read_csv(r"/content/crop_yield.csv")

# If 'Year' column is not datetime, create a date column
crop_yield['date'] = pd.to_datetime(crop_yield['Year'].astype(str) + "-01-01", errors='coerce')

# Keep only necessary columns
crop_yield_small = crop_yield[['date', 'Crop_Yield (kg/ha)']]

# Ensure crop_yield date column is datetime (just in case)
crop_yield_small['date'] = pd.to_datetime(crop_yield_small['date'], errors='coerce')

# Merge
df = df.merge(crop_yield_small, on='date', how='left')

# Save
df.to_csv("merged_final.csv", index=False)

print("Merged Step 4 (Crop Yield) complete. Final dataset saved as merged_final.csv")

/tmp/ipython-input-102457592.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crop_yield_small['date'] = pd.to_datetime(crop_yield_small['date'], errors='coerce')
